In [5]:
import concurrent.futures
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo, commonallplayers
import pandas as pd
import time
import requests

# Function to fetch all players
def fetch_all_players(retries=5, delay=10):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Referer': 'https://stats.nba.com/',
        'Origin': 'https://stats.nba.com',
        'Connection': 'keep-alive',
    })
    for attempt in range(retries):
        try:
            all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1, headers=session.headers).get_data_frames()[0]
            print("Fetched all players successfully.")
            return all_players
        except Exception as e:
            print(f"Attempt {attempt + 1} - An error occurred while fetching all players: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
    return pd.DataFrame()  # Return an empty DataFrame if all retries fail

# Fetch all players for the current season
all_players = fetch_all_players()

# Fetch all players' information
def fetch_player_info(player_id):
    try:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
        return {
            'player_id': player_id,
            'first_name': player_info.loc[0, 'FIRST_NAME'],
            'last_name': player_info.loc[0, 'LAST_NAME'],
            'team': player_info.loc[0, 'TEAM_NAME'],
            'position': player_info.loc[0, 'POSITION'],
            'height': player_info.loc[0, 'HEIGHT'],
            'weight': player_info.loc[0, 'WEIGHT']
        }
    except Exception as e:
        print(f"An error occurred while fetching player info for player ID {player_id}: {e}")
        return None

# Function to fetch player stats
def fetch_player_stats(player_id, season):
    try:
        print(f"Fetching stats for player ID: {player_id}")
        career_stats = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
        season_stats = career_stats[career_stats['SEASON_ID'] == f'2{season[-2:]}']
        return season_stats
    except Exception as e:
        print(f"An error occurred while fetching stats for player ID {player_id}: {e}")
        return pd.DataFrame()

# Get all unique player IDs from the fetched players
player_ids = all_players['PERSON_ID'].unique()

# Initialize a DataFrame to store all player information and stats
all_player_data = []

# Use ThreadPoolExecutor to fetch player info and stats in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    # Fetch player info
    player_infos = list(executor.map(fetch_player_info, player_ids))
    player_infos = [info for info in player_infos if info is not None]  # Filter out None results

    # Fetch player stats
    for player_info in player_infos:
        player_id = player_info['player_id']
        player_stats = fetch_player_stats(player_id)
        if not player_stats.empty:
            # Append player info and stats to the list
            for stat in player_stats.itertuples(index=False):
                player_data = {**player_info, **stat._asdict()}
                all_player_data.append(player_data)

# Convert the list of player data to a DataFrame
all_player_data_df = pd.DataFrame(all_player_data)

# Select and display the relevant columns
relevant_columns = ['first_name', 'last_name', 'team', 'position', 'height', 'weight', 'PTS', 'REB', 'AST', 'STL', 'BLK']
all_player_data_df = all_player_data_df[relevant_columns]
display(all_player_data_df.head())

# Save the combined player data to a CSV file
all_player_data_df.to_csv('all_player_data_2022_2023.csv', index=False)
print("Player data saved successfully!")


    PERSON_ID DISPLAY_LAST_COMMA_FIRST DISPLAY_FIRST_LAST  ROSTERSTATUS  \
10    1630173        Achiuwa, Precious   Precious Achiuwa             1   
22     203500            Adams, Steven       Steven Adams             1   
24    1628389             Adebayo, Bam        Bam Adebayo             1   
29    1630534            Agbaji, Ochai       Ochai Agbaji             1   
42    1630583            Aldama, Santi       Santi Aldama             1   

    FROM_YEAR  TO_YEAR        PLAYERCODE       PLAYER_SLUG     TEAM_ID  \
10       2020     2024  precious_achiuwa  precious_achiuwa  1610612752   
22       2013     2024      steven_adams      steven_adams  1610612745   
24       2017     2024       bam_adebayo       bam_adebayo  1610612748   
29       2022     2024      ochai_agbaji      ochai_agbaji  1610612761   
42       2021     2024      santi_aldama      santi_aldama  1610612763   

   TEAM_CITY  TEAM_NAME TEAM_ABBREVIATION  TEAM_CODE  TEAM_SLUG  \
10  New York     Knicks              